In [432]:
import pandas as pd
import numpy as np
from collections import Counter
import copy
from sklearn.model_selection import train_test_split
import tensorflow as tf
import keras
from tensorflow.keras import layers
import keras.backend as K
from keras.regularizers import l2

### Files

In [433]:
q1=pd.read_csv('/content/q1.csv')
q2=pd.read_csv('/content/q2.csv')
q3=pd.read_csv('/content/q3.csv')

In [434]:
#Merge all files
q=pd.concat([q1,q2,q3],axis=0)
del q1,q2,q3
q.head()

,Patient Number;Billing Year;Billing Month;Diagnosis;Birth Year;Sex
0,10000784;2015;6;H10.8;1999;F
1,10000784;2015;6;K02.1;1999;F
2,10000784;2015;6;Z01.2;1999;F
3,10000784;2015;7;L70.0;1999;F
4,10000784;2015;8;K10.2;1999;F


### Preprocesing

In [435]:
#Sparse features
tempo=[]
for elmt in q.values:
  if(len(elmt[0].split(";"))==6):
      tempo.append(elmt[0].split(";"))

In [436]:
data=pd.DataFrame(data=tempo).rename(columns={0:'Patient number',1:'Billing year',2:'Billing month',3:'Diagnosis',4:'Birth year',5:'Sex'})
del tempo
data.dropna(inplace=True)
data.head()

,Patient number,Billing year,Billing month,Diagnosis,Birth year,Sex
0,10000784,2015,6,H10.8,1999,F
1,10000784,2015,6,K02.1,1999,F
2,10000784,2015,6,Z01.2,1999,F
3,10000784,2015,7,L70.0,1999,F
4,10000784,2015,8,K10.2,1999,F


In [437]:
#Extract list of patients whith F33.2 and/or F33.2 psychiatry illness
list_patient=list(set(data.loc[(data.Diagnosis=='F33.2') | (data.Diagnosis=='F32.0'),:]['Patient number'].values))

In [438]:
#Keep only patient whith F33.2 and/or F33.2 psychiatry illness
data=data.loc[data['Patient number'].isin(list_patient),:]
data.head()

,Patient number,Billing year,Billing month,Diagnosis,Birth year,Sex
467,10001443,2015,6,D50,1995,F
468,10001443,2015,7,F33.2,1995,F
469,10001443,2015,10,J06,1995,F
470,10001443,2015,11,F33.2,1995,F
471,10001443,2015,12,H50.3,1995,F


In [439]:
#Encode diagnosis
data["Diagnosis_coded"] = data["Diagnosis"].astype('category')
data["Diagnosis_coded"] = data["Diagnosis_coded"].cat.codes
data.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Patient number,Billing year,Billing month,Diagnosis,Birth year,Sex,Diagnosis_coded
467,10001443,2015,6,D50,1995,F,517
468,10001443,2015,7,F33.2,1995,F,1008
469,10001443,2015,10,J06,1995,F,1936
470,10001443,2015,11,F33.2,1995,F,1008
471,10001443,2015,12,H50.3,1995,F,1528


In [66]:
#Dictionary diagnois->diagnosis_coded
diagnosis_code_dict={key:val for key,val in zip(data.Diagnosis.values,data.Diagnosis_coded.values)}

In [67]:
#Group diagnois by patient
patient_diagnosis=data[['Patient number','Diagnosis']].groupby('Patient number')['Diagnosis'].apply(set).apply(list)
patient_diagnosis=pd.DataFrame({'Patient_number':patient_diagnosis.index, 'list_diagnosis':patient_diagnosis.values})
patient_diagnosis.head()

,Patient_number,list_diagnosis
0,10000928,"[K10.2, F33.11, J01.9, F33.2, F32.3, M65, F32...."
1,10001443,"[J06, S90.1, D50, N64.5, A09, G44.8, N84.0, N3..."
2,10001706,"[Z01.4, K02.2, R10.4, Z34.0, F32.10, F33.2, F3..."
3,10002015,"[D50, R10.4, S71.7, F41.9, Z01.7, F32.10, B00,..."
4,10002177,"[M41.2, F33.2, Z00.4, J30, F32.2, Z76.0, J03.9..."


In [68]:
#Calculate the average length of diagnosis per patient
tempo=[]
for lst in patient_diagnosis.list_diagnosis.values:
  tempo.append(len(lst))

diag_length=np.mean(np.array(tempo))
print(f'Average length of diagnosis per patient->{diag_length}')
del tempo

Average length of diagnosis per patient->21.016866330390922


In [69]:
#Calculate the occurence of each diagnosis
diag_occu_dict=Counter(data.Diagnosis.values)

In [70]:
#Create data frame with target_diagnosis column
tempo=[]
for pt,lst in zip(patient_diagnosis.Patient_number.values,patient_diagnosis.list_diagnosis.values):
  if ('F32.0' in lst) and ('F33.2' in lst):
    x=copy.deepcopy(lst)
    #Since there are more 'F33.2' classes than 'F32.0' we keep 'F32.0' and remove 'F33.2' for patient having both
    x.remove('F32.0')
    x.remove('F33.2')
    tempo.append([pt,x,'F32.0'])
  elif(('F32.0' in lst)):
    x=copy.deepcopy(lst)
    x.remove('F32.0')
    tempo.append([pt,x,'F32.0'])
  elif(('F33.2' in lst)):
    x=copy.deepcopy(lst)
    x.remove('F33.2')
    tempo.append([pt,x,'F33.2'])


data=pd.DataFrame(data=tempo).rename(columns={0:'Patient_number',1:'List_diagnosis',2:'Target_diagnosis'})
del tempo
data.head(5)

,Patient_number,List_diagnosis,Target_diagnosis
0,10000928,"[K10.2, F33.11, J01.9, F32.3, M65, F32.2, F19....",F33.2
1,10001443,"[J06, S90.1, D50, N64.5, A09, G44.8, N84.0, N3...",F33.2
2,10001706,"[Z01.4, K02.2, R10.4, Z34.0, F32.10, F32.2, O8...",F33.2
3,10002015,"[D50, R10.4, S71.7, F41.9, Z01.7, F32.10, B00,...",F33.2
4,10002177,"[M41.2, Z00.4, J30, F32.2, Z76.0, J03.9, F61.0...",F33.2


In [73]:
#Dictionary of target_diagnois
target_dic={}
target_dic["F33.2"]=0
target_dic["F32.0"]=1

In [74]:
#Create a new dataframe where doagnosis in List_diagnois are those having a high occurence and padd them if needed.
tempo=[]
for pt,lst_diag,trgt in zip(data.Patient_number.values,data.List_diagnosis.values,data.Target_diagnosis.values):
 

  tempo_occurence_list=list(map(lambda x: diag_occu_dict[x],lst_diag))

  # top_idx=np.argsort(tempo_occurence_list)[-2:]
  # print(top_idx)
  
  if(len(lst_diag)<int(diag_length)):
    diag_coded_list=list(map(lambda x: diagnosis_code_dict[x],lst_diag))+[0]*(int(diag_length)-len(lst_diag))
    tempo.append([pt,diag_coded_list,target_dic[trgt]])
  else:
    tempo_occurence_list=list(map(lambda x: diag_occu_dict[x],lst_diag))
    top_idx=np.argsort(tempo_occurence_list)[-int(diag_length):]
    x=np.array(lst_diag)[top_idx]
    x=list(map(lambda x: diagnosis_code_dict[x],x))
    tempo.append([pt,x,target_dic[trgt]])


data=pd.DataFrame(data=tempo).rename(columns={0:'Patient_number',1:'List_diagnosis_coded',2:'Target_diagnosis'})
del tempo
data.head(5)

,Patient_number,List_diagnosis_coded,Target_diagnosis
0,10000928,"[2140, 1007, 1920, 998, 2897, 997, 898, 19, 10...",0
1,10001443,"[3326, 3242, 1255, 517, 4539, 19, 4542, 4466, ...",0
2,10001706,"[4411, 2095, 3604, 4542, 995, 997, 3464, 135, ...",0
3,10002015,"[79, 165, 1920, 3601, 77, 4414, 517, 1914, 446...",0
4,10002177,"[2808, 4402, 1983, 997, 4655, 1928, 1125, 1070...",0


In [76]:
#Class distribution
data.Target_diagnosis.value_counts()

0    4319
1    2025
Name: Target_diagnosis, dtype: int64

### Prepare input/output

In [86]:
X_diag=np.array([lst for lst in data.List_diagnosis_coded.values]).reshape(len(data),int(diag_length))
y=np.array([lst for lst in data.Target_diagnosis.values])
X_diag.shape,y.shape

((6344, 21), (6344,))

In [268]:
#Split data training/test
X_diag_train,X_diag_test,y_train,y_test=train_test_split(X_diag,y,test_size=0.1,stratify=y)
X_diag_train.shape,X_diag_test.shape,y_train.shape,y_test.shape

((5709, 21), (635, 21), (5709,), (635,))

### Model

In [412]:
class Deepsy(keras.Model):
    
    def __init__(self,voc_size,embedding_size,input_length):
        super(DeppPsy, self).__init__()
        
        self.voc_size=voc_size
        self.embedding_size=embedding_size
        self.input_length=input_length

        self.embedding = layers.Embedding(self.voc_size,self.embedding_size,input_length=self.input_length,mask_zero=True)
        self.conv1d_diag_1=layers.Conv1D(256,3,padding='same',activation='relu')
        self.max_pooling1d_diag_1=layers.MaxPooling1D(pool_size=5, strides=None, padding='valid')
        self.conv1d_diag_2=layers.Conv1D(30,5,padding='same',activation='relu')
        self.max_pooling1d_diag_2=layers.MaxPooling1D(pool_size=3, strides=None, padding='valid')
        self.flatten=layers.Flatten()
        self.lstm=layers.LSTM(100,dropout=0.5,return_sequences=False)
        self.dense_1=layers.Dense(20,activation='tanh')
        self.final_dense=layers.Dense(1,activation='sigmoid')

        self.embedding_dropout=layers.Dropout(0.9)
        self.conv_dropout=layers.Dropout(0.9)
        self.dense_dropout=layers.Dropout(0.5)



    def call(self, inputs,training=None):
        lst_diag=inputs

        lst_diag=(self.embedding(lst_diag))

        if training:
          lst_diag=self.embedding_dropout(lst_diag)

        lst_diag=tf.reduce_sum(lst_diag,1)

        if training:
          lst_diag=self.dense_dropout(lst_diag)

        lst_diag=self.dense_1(lst_diag)

        if training:
          lst_diag=self.dense_dropout(lst_diag)

        lst_diag=self.final_dense(lst_diag)


        return lst_diag

### Training

In [238]:
# weighted binary crossentropy
def weighted_binary_crossentropy(alpha_factor=2.0):

    def weighted_binary_crossentropy(y_true, y_pred):
        
        b_ce = tf.keras.losses.BinaryCrossentropy(from_logits=False)
        loss = b_ce(y_true, y_pred)

        weight_vector = tf.where(K.equal(y_true, 1), alpha_factor, 1)
        weighted_b_ce = weight_vector * loss

        return K.mean(weighted_b_ce)

    return weighted_binary_crossentropy

In [429]:
#Configuration
bc=keras.losses.BinaryCrossentropy(from_logits=False)
epoch=25
batch_size=100

In [430]:
#Training
model=Deepsy(len(diagnosis_code_dict),50,X_diag_train.shape[1])
model.compile(loss=weighted_binary_crossentropy(), optimizer='adam', metrics=['accuracy'])
model.fit(X_diag_train, y_train, epochs=epoch, batch_size=batch_size, verbose=2)

Epoch 1/25
58/58 - 1s - loss: 2.8344 - accuracy: 0.5773 - 1s/epoch - 21ms/step
Epoch 2/25
58/58 - 0s - loss: 2.6703 - accuracy: 0.6248 - 364ms/epoch - 6ms/step
Epoch 3/25
58/58 - 0s - loss: 2.5823 - accuracy: 0.6301 - 365ms/epoch - 6ms/step
Epoch 4/25
58/58 - 0s - loss: 2.5168 - accuracy: 0.6481 - 368ms/epoch - 6ms/step
Epoch 5/25
58/58 - 0s - loss: 2.4560 - accuracy: 0.6644 - 364ms/epoch - 6ms/step
Epoch 6/25
58/58 - 0s - loss: 2.4129 - accuracy: 0.6721 - 372ms/epoch - 6ms/step
Epoch 7/25
58/58 - 0s - loss: 2.3830 - accuracy: 0.6821 - 370ms/epoch - 6ms/step
Epoch 8/25
58/58 - 0s - loss: 2.3393 - accuracy: 0.6889 - 355ms/epoch - 6ms/step
Epoch 9/25
58/58 - 0s - loss: 2.2946 - accuracy: 0.6945 - 364ms/epoch - 6ms/step
Epoch 10/25
58/58 - 0s - loss: 2.2416 - accuracy: 0.7073 - 362ms/epoch - 6ms/step
Epoch 11/25
58/58 - 0s - loss: 2.2419 - accuracy: 0.7024 - 369ms/epoch - 6ms/step
Epoch 12/25
58/58 - 0s - loss: 2.2039 - accuracy: 0.7117 - 371ms/epoch - 6ms/step
Epoch 13/25
58/58 - 0s - lo

In [431]:
#Test
model.evaluate(X_diag_test, y_test, verbose=1)

20/20 [==============================] - 0s 2ms/step - loss: 2.0921 - accuracy: 0.7559


[2.0921289920806885, 0.7559055089950562]